In [137]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

In [138]:
from statsforecast import StatsForecast
from statsforecast.models import AutoARIMA

In [139]:
stock = "MSFT"
period = "5y"

In [140]:
import yfinance

data = yfinance.download(stock,period=period)
data

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-08,88.199997,88.580002,87.599998,88.279999,83.134613,22113000
2018-01-09,88.650002,88.730003,87.860001,88.220001,83.078087,19484300
2018-01-10,87.860001,88.190002,87.410004,87.820000,82.701401,18652200
2018-01-11,88.129997,88.129997,87.239998,88.080002,82.946243,17808900
2018-01-12,88.669998,89.779999,88.449997,89.599998,84.377647,24271500
...,...,...,...,...,...,...
2022-12-29,235.649994,241.919998,235.649994,241.009995,241.009995,19770700
2022-12-30,238.210007,239.960007,236.660004,239.820007,239.820007,21930800
2023-01-03,243.080002,245.750000,237.399994,239.580002,239.580002,25740000


In [141]:
data = data[["Adj Close"]]
data.reset_index(inplace=True)
data["unique_id"] = stock
data.rename(columns={"Date":"ds","Adj Close":"y"},inplace=True)
data = data [["unique_id","ds","y"]]
data["trend"] = np.arange(1,len(data)+1)
data

,unique_id,ds,y,trend
0,MSFT,2018-01-08,83.134613,1
1,MSFT,2018-01-09,83.078087,2
2,MSFT,2018-01-10,82.701401,3
3,MSFT,2018-01-11,82.946243,4
4,MSFT,2018-01-12,84.377647,5
...,...,...,...,...
1253,MSFT,2022-12-29,241.009995,1254
1254,MSFT,2022-12-30,239.820007,1255
1255,MSFT,2023-01-03,239.580002,1256
1256,MSFT,2023-01-04,229.100006,1257


In [142]:
train = data[data.ds<"2022-01-01"]
test = data[data.ds>="2022-01-01"]
print(test.head())
print(train)

     unique_id         ds           y  trend
1004      MSFT 2022-01-03  331.642456   1005
1005      MSFT 2022-01-04  325.955750   1006
1006      MSFT 2022-01-05  313.442993   1007
1007      MSFT 2022-01-06  310.966187   1008
1008      MSFT 2022-01-07  311.124756   1009
     unique_id         ds           y  trend
0         MSFT 2018-01-08   83.134613      1
1         MSFT 2018-01-09   83.078087      2
2         MSFT 2018-01-10   82.701401      3
3         MSFT 2018-01-11   82.946243      4
4         MSFT 2018-01-12   84.377647      5
...        ...        ...         ...    ...
999       MSFT 2021-12-27  339.270996   1000
1000      MSFT 2021-12-28  338.082092   1001
1001      MSFT 2021-12-29  338.775635   1002
1002      MSFT 2021-12-30  336.170044   1003
1003      MSFT 2021-12-31  333.197876   1004

[1004 rows x 4 columns]


In [143]:
test = test.drop("y",axis=1,errors="ignore")
test.reset_index(drop=True,inplace=True)
test

,unique_id,ds,trend
0,MSFT,2022-01-03,1005
1,MSFT,2022-01-04,1006
2,MSFT,2022-01-05,1007
3,MSFT,2022-01-06,1008
4,MSFT,2022-01-07,1009
...,...,...,...
249,MSFT,2022-12-29,1254
250,MSFT,2022-12-30,1255
251,MSFT,2023-01-03,1256
252,MSFT,2023-01-04,1257


In [144]:
model = StatsForecast(
    df = train,
    models=[AutoARIMA(season_length=365)],
    freq="D",
    n_jobs=1,
    verbose=True
)

In [145]:
predictions = model.forecast(h=254,X_df=test)
predictions

Forecast:   0%|          | 0/1 [00:00<?, ?it/s]

ValueError: xreg is rank deficient